In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from torch.func import vmap, jacrev
from tqdm import tqdm
import os
import random
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import Callback
import math
from pydmd import DMD
from sklearn.preprocessing import MinMaxScaler
import warnings

In [2]:
class ResidualFlow(nn.Module):
    def __init__(self, dim, hidden_dim, n_layers, input_dim=0, dropout=0, LDJ=False, block_id=0):
        super().__init__()
        self.dim = dim
        self.input_dim = input_dim
        self.LDJ = LDJ
        self.block_id = block_id

        flip = (block_id % 2 == 0)
        self.flow = Flow(dim, hidden_dim, flip=flip)

    def forward(self, x, reverse=False):
        x_e = x
        if not reverse:
            y = self.flow(x_e, reverse=False)
            logdet = 0
            return y, logdet
        else:
            y = self.flow(x_e, reverse=True)
            return y

class Flow(nn.Module):
    def __init__(self, in_channel, hidden_dim, flip=False):
        super().__init__()
        self.coupling = AffineCoupling(in_channel, hidden_dim, flip)

    def forward(self, x, reverse=False):
        return self.coupling(x, reverse)

class AffineCoupling(nn.Module):
    def __init__(self, dim, hidden_dim, flip=False):
        super().__init__()
        self.dim = dim
        self.hidden_dim = hidden_dim
        self.flip = flip

        self.split_idx = dim // 2
        self.rest_dim = dim - self.split_idx

        self.net = nn.Sequential(
            nn.Linear(self.split_idx, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, self.rest_dim * 2)
        )

    def forward(self, x, reverse=False):
        if self.flip:
            x2, x1 = torch.split(x, [self.rest_dim, self.split_idx], dim=-1)
        else:
            x1, x2 = torch.split(x, [self.split_idx, self.rest_dim], dim=-1)

        h = self.net(x1)
        s, t = torch.chunk(h, 2, dim=-1)
        s = torch.tanh(s)

        if not reverse:
            y2 = x2 * torch.exp(s) + t
        else:
            y2 = (x2 - t) * torch.exp(-s)

        if self.flip:
            return torch.cat([y2, x1], dim=-1)
        else:
            return torch.cat([x1, y2], dim=-1)
    
class InvertibleNN(nn.Module):
    def __init__(self, dim, hidden_dim, n_blocks, n_layers, input_dim=0, dropout=0, LDJ=False):
        super(InvertibleNN, self).__init__()
        self.dim = dim
        self.hidden_dim = hidden_dim
        self.n_blocks = n_blocks
        self.n_layers = n_layers
        self.input_dim = input_dim
        self.blocks = nn.ModuleList([ResidualFlow(self.dim, self.hidden_dim, self.n_layers, self.input_dim, dropout, LDJ, block_id=i) for i in range(self.n_blocks)])
    
    def forward(self, x, u=None, reverse=False):
        if not reverse:
            ldj_total = 0
            for block in self.blocks:
                x, ldj = block(x, reverse)
                ldj_total += ldj
            return x, ldj_total
        else:
            for block in reversed(self.blocks):
                x = block(x, reverse)
            return x
    
class CombinedNetwork(nn.Module):
    def __init__(self, inn_model, input_dim, lifted_dim):
        super(CombinedNetwork, self).__init__()
        self.input_dim = input_dim
        self.inn_model = inn_model  
        self.lifted_dim = lifted_dim
    
    def forward(self, x, u=None, reverse=False):
        x = x.float()
        if not reverse:
            zero_pad = torch.zeros(x.shape[0], x.shape[1], self.lifted_dim, device=x.device)
            x = torch.cat((x, zero_pad), dim=-1)
            x, ldj = self.inn_model(x, u, reverse)
            return x, ldj
        else:
            x = self.inn_model(x, u, reverse)
            x = x[:, :self.input_dim]
            return x

In [3]:
def dmd(model, X, rank):
    GX_pred_list = []
    GX_list = []
    X_list = []
    GX, _ = model(X)
    for i in range(X.shape[0]):
        GX_temp = GX[i, :, :].T
        dmd = DMD(svd_rank=rank, exact=True, sorted_eigs='abs')
        dmd.fit(GX_temp.cpu().detach().numpy())
        GX_pred = dmd.reconstructed_data.real
        GX_pred = np.array(GX_pred, dtype=np.float32)
        GX_pred = torch.from_numpy(GX_pred).cuda()
        GX_pred_list.append(GX_pred)
        GX_list.append(GX_temp)
        X_list.append(X[i, :, :].T)
    GX_pred = torch.cat(GX_pred_list, dim=-1)
    GX = torch.cat(GX_list, dim=1)
    X = torch.cat(X_list, dim=-1)

    return GX, GX_pred, X

In [4]:
class TrainModel(pl.LightningModule):
    def __init__(self, model, rank, learning_rate=1e-3, lamb=1, path="model_checkpoint_NP"):
        super(TrainModel, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = nn.MSELoss()
        self.best_val_loss = float('inf') 
        self.validation_outputs = []
        self.lamb = lamb
        self.train_losses = []
        self.rank = rank
        self.path = path+'.ckpt'

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        X_batch = batch[0]
        GY, GY_pred, Y = dmd(self.model, X_batch, self.rank)
        Y_pred = self.model(GY_pred.T, reverse=True)
        loss_lin = self.criterion(GY, GY_pred)
        loss_recons = self.criterion(Y.T, Y_pred)

        loss = loss_lin + self.lamb * loss_recons
        self.log('train_loss', loss, on_step=True, on_epoch=False, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        Z_batch = batch[0]
        Z1, Z_pred, Z = dmd(self.model, Z_batch, self.rank)
        Z_pred = self.model(Z_pred.T, reverse=True)
        valid_loss = self.criterion(Z_pred, Z.T)

        self.validation_outputs.append(valid_loss)
        self.log('val_loss', valid_loss)
        return valid_loss

    def test_step(self, batch, batch_idx):
        Z_batch = batch[0]
        Z1, Z_pred, Z = dmd(self.model, Z_batch, self.rank)
        Z_pred = self.model(Z_pred.T, reverse=True)
        test_loss = self.criterion(Z_pred[:, :self.model.inn_model.input_dim], Z.T[:, :self.model.inn_model.input_dim])

        self.log('test_loss', test_loss)
        return test_loss
    
    def on_fit_start(self):
        if self.trainer.is_global_zero: 
            if os.path.exists("loss_log.txt"):
                os.remove("loss_log.txt")
            if os.path.exists(self.path):
                os.remove(self.path)
    
    def on_train_epoch_end(self):
        if self.trainer.is_global_zero:
            avg_train_loss = self.trainer.callback_metrics.get("train_loss")
            if avg_train_loss is not None:
                self.train_losses.append(avg_train_loss.item()) 
                print(f"Epoch {self.current_epoch}: Average Training Loss = {avg_train_loss.item()}")

    def on_validation_epoch_end(self):
        avg_val_loss = torch.stack(self.validation_outputs).mean()  
        self.log('avg_val_loss', avg_val_loss)
        self.validation_outputs.clear()
        print(f"Validation loss: {avg_val_loss}")
        with open("loss_log.txt", "a") as f:
            f.write(f"{avg_val_loss.item()}\n")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, eps=1e-08,
                                            weight_decay=0)
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=1,
            gamma=0.99
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",  
            },
            "gradient_clip_val": 1.0, 
            "gradient_clip_algorithm": "norm",
        }

In [5]:
dim = 2
hidden_dim = 20 
input_dim = 1
n_blocks = 3  
n_layers = 1
n_feature = 8
rank = 5
batch_size = 512
n_train = 10000
n_valid = 1000
n_test = 1000
dropout = 0
num_epochs = 100 
lamb = 0
learning_rate = 1e-3  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
X_train = pd.read_csv('Non_X_train.csv', header=None).values
X_valid = pd.read_csv('Non_X_valid.csv', header=None).values
X_test = pd.read_csv('Non_X_test.csv', header=None).values
U_train = pd.read_csv('Non_U_train.csv', header=None).values
U_valid = pd.read_csv('Non_U_valid.csv', header=None).values
U_test = pd.read_csv('Non_U_test.csv', header=None).values

length = X_train.shape[1] // n_train
HX_train = []
HU_train = []
for i in range(n_train):
    HX_train.append(X_train[:, i*length:(i+1)*length])
    HU_train.append(U_train[:, i*length:(i+1)*length])
HX_train = np.stack([HX_train[idx].T for idx in range(n_train)], axis=0)
HU_train = np.stack([HU_train[idx].T for idx in range(n_train)], axis=0)
HX_valid = []
HU_valid = []
for i in range(n_valid):
    HX_valid.append(X_valid[:, i*length:(i+1)*length])
    HU_valid.append(U_valid[:, i*length:(i+1)*length])
HX_valid = np.stack([HX_valid[idx].T for idx in range(n_valid)], axis=0)
HU_valid = np.stack([HU_valid[idx].T for idx in range(n_valid)], axis=0)
H_train = np.concatenate([HX_train, HU_train], axis=-1)
H_valid = np.concatenate([HX_valid, HU_valid], axis=-1)
train_dataset = TensorDataset(torch.tensor(H_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_dataset = TensorDataset(torch.tensor(H_valid, dtype=torch.float32))
valid_loader = DataLoader(valid_dataset, batch_size=99999, shuffle=True, num_workers=8, pin_memory=True)

In [ ]:
warnings.filterwarnings("ignore")
path = "model_checkpoint_NP_flowdmd"
checkpoint_callback = ModelCheckpoint(
    monitor="avg_val_loss",   
    dirpath="./", 
    filename=path, 
    save_top_k=1, 
    mode="min",    
)
inn_model = InvertibleNN(dim=dim+input_dim+n_feature, hidden_dim=hidden_dim, n_blocks=n_blocks, n_layers=n_layers, input_dim=input_dim, dropout=dropout, LDJ=lamb>0)
model = CombinedNetwork(inn_model=inn_model, input_dim=dim+input_dim, lifted_dim=n_feature)
lightning_model = TrainModel(model=model, rank=rank, learning_rate=learning_rate, lamb=lamb, path=path)
trainer = pl.Trainer(accelerator="gpu", devices=4, strategy="ddp_notebook", max_epochs=num_epochs, callbacks=[checkpoint_callback])

trainer.fit(lightning_model, train_loader, valid_loader)

In [9]:
inn_model = InvertibleNN(dim=dim+input_dim+n_feature, hidden_dim=hidden_dim, n_blocks=n_blocks, n_layers=n_layers, input_dim=input_dim, dropout=dropout, LDJ=lamb>0)
model = CombinedNetwork(inn_model=inn_model, input_dim=dim, lifted_dim=n_feature)
path = "model_checkpoint_NP_flowdmd.ckpt"
lightning_model = TrainModel.load_from_checkpoint(path, model=model, rank=rank, learning_rate=learning_rate, map_location="cpu")
trainer = pl.Trainer(accelerator="gpu", devices=4, strategy="ddp_notebook", max_epochs=num_epochs)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
length = X_test.shape[1] // n_test
HX_test = []
HU_test = []
for i in range(n_test):
    HX_test.append(X_test[:, i*length:(i+1)*length])
    HU_test.append(U_test[:, i*length:(i+1)*length])
HX_test = np.stack([HX_test[idx].T for idx in range(n_test)], axis=0)
HU_test = np.stack([HU_test[idx].T for idx in range(n_test)], axis=0)
H_test = np.concatenate([HX_test, HU_test], axis=-1)
test_dataset = TensorDataset(torch.tensor(H_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=9999, shuffle=True)

In [11]:
warnings.filterwarnings("ignore")
trainer.test(lightning_model, dataloaders=test_loader)

You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable

Testing DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  0.66it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.0030282342340797186
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.0030282342340797186}]